<a href="https://colab.research.google.com/github/russburns/Laframboise1966Code/blob/main/ADJUST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
        SUBROUTINE ADJUST(NEND,MAD,MEW,QT1,QT2,KWIT, ACCY, YUSE,YINY)
C       SUBROUTINE ADJUST MONITORS CONVERGENCE OF THE CALCULATIONS AND
C           TAKES CORRECTIVE ACTION WHEN NECESSARY

C       IF KWIT=1 ADJUST TAKES NO ACTION
C       IF KWIT=2 ADJUST DAMPS ANY DIVERGENT OSCILLATIONS
C       IF KTWIT = 3 ADJUST ALSO ENDS EXECUTION WHEN ACCURACY OF RESULTS
C           IS SUFFICIENT

        DIMENSION YCHEK(10),DCHEK(10),ZCHEK(10)
        DIMENSION X(401),XSQ(401),S(401),DXDS(401),ROP(401),SCOT(401),
    1       COOK(401),XI(401),DXIDS(401),ETA(401),ETAPS(401),ETANG(401),
    2       RHO(401),OMGAG(401),BETAG(401),ALFAG(401),PSIG(401),EPSG(401),
    3       Y(401),Z(401),SH(401)
        COMMON X,XSQ,S,DXDX,ROP,SCOT,COOK,XI,DXIDS,ETA,ETAPS,ETANG,RHO,
    1       OMGAG,BETAG,ALFAG,PSIG,EPSG,Y,Z,SH
        COMMON PI,SQTPI,VIPI,SAY,MODE,M,MP,DELTS,GAMMA,PI3,PI6,PI7,YPOS,
    1       YNEG,NPRINT,KT1,KT2,LL,KEND
        COMMON LINK,BETH,EXY,EDY,N2,N22,NW,SW,SWA,BETAW,BETAWA,MACK,MIKE,
    1       SCRIT,SCRITA,LK,LKA,CRISS,CROSS,YST,AMU,THETA,KBD,MCD,MZET,HALT

        IF(KWIT-1)228,228,21

C       IF KCHEK = 1 TO 9, STORE YCHEK(KCHEK) AND RETURN.

    21  KCHEK=XMODF(KEND,10)
        IF(KCHEK)251,250,251
    250 KCHEK=10
    251 YCHKE(KCHEK)=YUSE
    232 IF(KCHEK-8)228,121,122
    121 A8=B8
        B8=C8
        C8=YCHEK(8)
        GO TO 228
    122 IF(KCHEK-10)124,234,228
    124 A9=B9
        B9=C9
        C9=YCHEK(9)
        GO TO 228
C       IF KCHECK = 10, STORE YCHEK(10) AND LOOK FOR OSCILLATIONS IN
C       YCHEK(1-10)

    234 A10=B10
        B10=C10
        C10=YCHEK(10)
        DO 235 I=2,10
    235 DCHEK(I)=YCHEK(I)-YCHEK(I-1)
        DO 236 I=2,9
        IF (DCHEK(I))237,237,238
    237 IF(DCHEK(I+1))123,236,236
    238 IF(DCHEK(I+1))236,236,123
    236 CONTINUE
        DO 240 I=2,9
    240 ZCHEK(I)=(ABSF(DCHEK(I))+ABSF(DCHEK(I+1)))/2.0
        SMCHEK=0.0
        DO 262 I=2,8
    262 SMCHEK=SMCHEK+ZCHEK(I+1)/ZCHEK(I)
        IF(SMCHEK-6.3)123,123,242
C       OSCILLATIONS HAVE BEEN OBSERVED. DECREASE MIXING AND RETURN.

    242 MEW=0
        MAD=2
    125 QT1=QT1*0.9
        QT2=QT2*0.9
        WRITE OUTPUT TAPE 6,127,KEND,QT1,QT2
    127 FORMAT(7H0, KEND=14,27H 0.9 DECREASE IN MIX. QT1=F10.7,5H QT2=
    1   F10.7)
    128 DO 247 I=1,MP
    247 COOK(I)=COOK(I)*0.9
        228

C       OSCILLATIONS FOUNO TO BE ABSENT OR DECREASING.

    123  IF(KWIT-2)228,228,132

    132 GO TO (280,281),MZET

    280 MEW=MEW+1
        IF(MEW-2)228,134,134

    281 YMIN=YCHEK(1)
        YMAX=YCHEK(1)
        DO 282 I=1,9
        YMIN=MIN1F(YMIN,YCHEK(I+1))
    282 YMAX=MAX1F(YMIN,YCHEK(I+1))
        IF(ABSF(2.0*(YMAX-YMIN)(YMAX+YMIN))-ACCY)283,283,228
    283 YINY=(C9+C10)/2.0
        GO TO 278

C       FIND OUT IF CONVERGENCE TOWARD AN ASYMPTOTIC RESULT IS YET
C       INDICATED. IF SO, GO TO 138

    134 RIO=(B10-C10)/(A10-B10)
        IF(RIO)228,228,137
    137 IF(RIO-1.0)138,228,228

C       ESTIMATE NUMBER OF ITERATIONS REMAINING BEFORE REQUIRED ACCURACY
C       IS ATTAINED. IF THIS EXCEEDS 40, GO TO 141. OTHERWISE, GO TO 140.

    138 YINY=A10-(A10-B10)/(1.0-RIO)
        ALPH=(C10-YINY)/(A10-YINY)
        FINS=20.0*(LOGF (ACCY*YINY/(C10-YINY))/LOGF(ALPH))
        KINS=FINS
        IF(KINS-40)140,140,141

C       CONVERGENCE IS TOO SLOW. IF MIXING FUNCTION HAS NOT ALREADY BEEN
C       DECREASED BECAUSE OF OSCILLATIONS, INCREASE IT.

    141 IF(KWIT-3)228,228,28
    28  GO TO (126,228),MAD

    126 FACT=1.111
        QTY=QT2*FACT
        QT2=MIN1F(QTY,0.7)
        FACT=FACT*QT2/QTY
        QTS=QT1
        ZIT=1.0-QT2
        ZAT=QT1*FACT
        QT1=MIN1F(ZIT,ZAT)
        MEW=0
        WRITE OUTPUT TAPE 6,142,KEND,QT1,QT2,C10,YINY
    142 FORMAT(4H0KND14,13H SLO CONV QT1F10.7,4H QT2F10.7,2H YIPE10.3,
    1   5H YINY1PE10.3)
    24  DO 143 I=1,MP
    143 COOK(I)=COOKIE(I,QT1,QT2)
        GO TO 228

    C   FIND OUT IF RESULT IS SUFFICIENTLY ACCURATE. IF SO, END EXECUTION
    C   IF NOT, GO TO 145.

    140 IJJ=1
        IF(ABSF(C10-YINY)-ABSF(ACCY*YINY))144,144,275
    144 IF(ABSF(C9-YINY)-ABSF(ACCY*YINY))146,146,275

    275 IJJ-2
    146 DO 266 I=1,MP
    266 Y(I)=(2.*(ETAPS(I)-ETANG(I)-ETA(I))/(ETAPS(I)-ETANG(I)+ETA(I)))**2
        TOT=0.0
        DO 267 I=1,M
    267 TOT=TOT+(Y(I+1)+Y(I))*(ROP(I+1)-ROP(I))*0.5
        AVGE=TOT/(ROP(MP)-1.0)
        IF(AVGE-0.0100)255,255,277

    277 IJJ=2
    255 GO TO (278,145),IJJ

    278 WRITE OUTPUT TAPE 6,147,KEND,YINY
    147 FORMAT(7H0 KEND=14,36H RESULT SUFFICIENTLY ACCURATE. YINY=1PE10.3)
    26  NEND=XMIN0F(KEND+8,NEND)
        GO TO 228

    145 GO TO (228,228,151,34),KWIT

C   FIND OUT IF THERE ARE OSCILLATIONS DECREASING TOO SLOWLY TO ALLOW
C   ATTAINMENT OF REQUIRED ACCURACY WITHIN 40 ITERATIONS.
C   IF SO, GO TO 159. IF NOT, GO TO 151.

    34  IF(A8-A9)148,148,149
    148 IF(A9-A10)151,151,150
    149 IF(A9-A10)150,150,151

    150 IF(B8-B9)152,152,153
    152 IF(B9-B10)151,151,154
    153 IF(B9-B10)154,154,151

    154 IF(C8-C9)155,155,156
    155 IF(C9-C10)151,151,157
    156 IF(C9-C10)157,157,151

    157 AMA=(ABSF(A8-A9)+ABSF(A9-A10))/2.0
        AMB=(ABSF(C8-C9)+ABSF(C9-C10))/2.0
        IF(AMB*(AMB/AMA)**2-ACCY*YINY)151,151,159

    151 WRITE OUTPUT TAPE 6,170,KEND,C10,YINY,KINS,AVGE
    170 FORMAY(7H0 KEND=14,3H Y=1PE10.3,6H YINY=1PE10.3,20H ESTD. CYCYLES T
    1   O END I3.6H AVGE=0PF8.4)
    270 GO TO 228

C       DECREASE MIXING FUNCTION.

    159 QT1=QT1*0.9
        QT2=QT2*0.9
        DO 171 I=1,MP
    171 COOK(I)=COOK(I)*0.9
        WRITE OUTPUT TAPE 6,172,KEND,QT1,QT2
    172 FORMAT( 5H0KEND 14,38H OSCILLATIONS DAMPING TOO SLOWLY. QT1=F10.7,
    1   5H QT2=F10.7)
    31  MEW=0
        MAD=2
        GO TO 228

    228 RETURN
        END